In [388]:
import duckdb
import lorem
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import time
import tqdm

# Source Column Data

In [342]:
NL = '\n'
TAB = '\t'
SOURCE_DATA_DIR = 'data/source'
GROUPS_DATA_DIR = 'data/groups'
sns.set_style("whitegrid")

In [3]:
con = duckdb.connect()

## Strings

In [346]:
def create_distinct_string_data(con, name, min_strlen, max_strlen, count):
    l = [c for c in list(lorem.text()) if c != '\r' ]
    source_string = ''
    
    offset_in_source = 0
    strlen_range = max_strlen - min_strlen
    
    unique_strings = set()
    while (len(unique_strings) < count):
        strlen = min_strlen + math.ceil(random.random() * strlen_range)
        
        if offset_in_source + strlen > len(source_string):
            random.shuffle(l)
            source_string = ''.join(l)
            offset_in_source = 0
        
        unique_strings.add(source_string[offset_in_source:offset_in_source + strlen])
        offset_in_source += strlen
    
    unique_strings = pd.DataFrame(list(unique_strings), columns=['c0'])
    con.execute(f"""
    COPY (SELECT * FROM unique_strings)
    TO '{SOURCE_DATA_DIR}/{name}_strings.parquet'
    """)

In [340]:
string_params = [
    ('tiny', 1, 1, 20),
    ('small', 4, 7, int(1e6)),
    ('medium', 8, 15, int(1e6)),
    ('large', 16, 31, int(1e6)),
    ('huge', 32, 63, int(1e6)),
]

for param in string_params:
    create_distinct_string_data(con, *param)

## Integral

In [347]:
limits = {
    'tinyint': (1 << 7) - 1,
    'smallint': (1 << 15) - 1,
    'integer': (1 << 31) - 1,
    'bigint': (1 << 63) - 1,
}

def create_distinct_integral_data(con, t, count):
    limit = limits[t]
    con.execute(f"""
    COPY (SELECT DISTINCT CAST(random() * {limit} - {limit} / 2 AS {t}) c0
          FROM range({count})
    ) TO '{SOURCE_DATA_DIR}/{t}s.parquet'
    """)

In [16]:
integral_params = [
    ('tinyint', 100),
    ('smallint', 10000),
    ('integer', int(1e8)),
    ('bigint', int(1e8)),
]

for param in integral_params:
    create_distinct_integral_data(con, *param)

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

# Group Generation

In [348]:
def generate_unique_groups(group_cols, group_count):
    group_unique_counts = []
    num_group_cols = len(group_cols)
    remaining_group_count = group_count
    for i in range(num_group_cols):
        assert(len(group_cols[i]) == 2)
        group_type = group_cols[i][0]
        group_weight = group_cols[i][1]
        assert(group_weight >= 0 and group_weight <= 1)
        remaining_group_cols = num_group_cols - i
        group_pow = (1 / remaining_group_cols) ** (1 - group_weight)
        assert(group_pow >= 0 and group_pow <= 1)
        group_unique_count = math.ceil(remaining_group_count ** group_pow)
        remaining_group_count /= group_unique_count
        group_unique_counts.append(group_unique_count)
    return f"""\rSELECT{TAB}row_number() OVER () AS group_id, *
               \rFROM{f',{NL}'.join([f"{TAB}(SELECT c0 AS c{i} FROM '{SOURCE_DATA_DIR}/{t}s.parquet' USING SAMPLE {c}) t{i}" for i, (t, c) in enumerate(zip([gc[0] for gc in group_cols], group_unique_counts))])}
               \rLIMIT{TAB}{group_count}
           """

In [364]:
def generate_dataset(name, group_cols, group_count, power, max_repetition, test_run=True):    
    # Create group occurrences
    scale = 1 - np.random.power(power, group_count)
    scale /= np.max(scale)
    counts_per_group = pd.DataFrame((max_repetition**(scale)).astype(int), columns=['count'])
    con.execute("""
    CREATE OR REPLACE TEMPORARY TABLE occurrences AS
    WITH cte AS (
        SELECT row_number() OVER () AS group_id, count AS group_count
        FROM counts_per_group
    )
    SELECT unnest([group_id for _ in range(group_count)]) AS group_id
    FROM cte
    """)
    
    if test_run:
        print(con.execute("""
        SELECT min(count) AS mi,
               max(count) AS ma,
               median(count) AS med,
               avg(count) AS avg,
               sum(count) AS s
        from counts_per_group
        """).fetchdf())
        
        print("Is the generated distribution OK?")
        a = input()
        if a == 'n':
            return
        elif a != 'y':
            print("Input must be y/n!")
            
    con.execute("""
    CREATE OR REPLACE TEMPORARY TABLE occurrences AS
    SELECT * FROM occurrences
    ORDER BY random()
    """)
    
    # Create groups
    q = generate_unique_groups(group_cols, group_count)
    con.execute(f"CREATE OR REPLACE VIEW groups AS ({q})")
    
    # Join them
    con.execute(f"""
    COPY (
        SELECT groups.* EXCLUDE (group_id)
        FROM groups
        JOIN occurrences
        USING (group_id)
    ) TO '{GROUPS_DATA_DIR}/{name}_{group_count}_{power}_{max_repetition}.parquet'
    """)
    

In [365]:
%%html
<style>
table {align:left;display:block}
</style>

We can choose from:
| table | width | unique |
|:---|:---:|:---:|
|`'tinyints'` | 1 | 100 |
|`'smallints'` | 2 | 10.000 |
|`'integers'` | 4 | 100.000.000 |
|`'bigints'` | 8 | 100.000.000 |
|`'tiny_strings'` | 1 | 20 |
|`'small_strings'` | 4-7 | 1.000.000 |
|`'medium_strings'` | 8-15 | 1.000.000 |
|`'large_strings'` | 16-31 | 1.000.000 |
|`'huge_strings'` | 32-63 | 1.000.000 |

In [368]:
# This mimics clickbench, but does not have the natural "bursty" behavior because we only do random uniform
group_cols = [
    ("integer", 0),
]
group_count = 17630976
power = 15
max_repetition = 29097

generate_dataset("clickbench_userid", group_cols, group_count, power, max_repetition)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   mi     ma  med       avg           s
0   1  29097  2.0  5.394694  95113726.0
Is the generated distribution OK?
y


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [391]:
group_cols = [
    ("integer", 0),
]

# Total row count (approx)
total_count = int(1e8)

# Random uniform
power = 1

# From 1k to 100M groups
for gce in tqdm.tqdm(range(3, 9)):
    group_count = int(float(f"1e{gce}"))
    max_repetition = int(total_count / group_count)
    generate_dataset("uniform_integers", group_cols, group_count, power, max_repetition, False)

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                              | 4/6 [00:05<00:03,  1.82s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                               | 5/6 [00:25<00:08,  8.16s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:41<00:00, 16.97s/it]


# Random Queries

In [375]:
before = time.time()
con.execute("""
WITH hits_per_user AS (
    SELECT UserID, count(*) AS count
    FROM '/Users/laurens/git/duckdb/clickbench/hits.parquet'
    GROUP BY UserID
)
select min(count) mi, max(count) ma, median(count) med, avg(count) avg, sum(count) s
FROM hits_per_user
""")
print(time.time() - before)
con.fetchdf()

1.2298848628997803


,mi,ma,med,avg,s
0,1,29097,2.0,5.671694,99997497.0


In [376]:
before = time.time()
con.execute(f"""
WITH hits_per_user AS (
    SELECT c0, count(*) AS count
    FROM '{GROUPS_DATA_DIR}/clickbench*.parquet'
    GROUP BY c0
)
select min(count) mi, max(count) ma, median(count) med, avg(count) avg, sum(count) s
FROM hits_per_user
""")
print(time.time() - before)
con.fetchdf()

1.7522392272949219


,mi,ma,med,avg,s
0,1,29097,2.0,5.394694,95113726.0
